In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)            
        )
        self.fc = nn.Linear(32*8*8, 10)
        
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        
        return x

In [4]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

transform_test = transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=False, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=False, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [5]:
# 앙상블 모델 만들기
num_models = 5
models = [CNN().to(device) for _ in range(num_models)]

criterion = nn.CrossEntropyLoss()
optimizers = [optim.Adam(model.parameters(), lr=0.001) for model in models]

In [ ]:
num_epochs = 2
for epoch in range(num_epochs):
    for model, optimizer in zip(models, optimizers):
        model.train()
        for imgs, labels in train_loader:
            imgs = imgs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            output = model(imgs)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()
            
    for model in models:
        model.eval()
        predictions = []
        targets = []
    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs = imgs.to(device)
            labels = labels.to(device)
            
            ensemble_outputs = torch.zeros((imgs.size(0), 10)).to(device) # 초기 앙상블 값
            for model in models:
                output = model(imgs)
                ensemble_outputs += output / num_models  # 초기값에 모델별 값 추가
            
            _, predicted = torch.max(ensemble_outputs.data, 1)
            
            predictions.extend(predicted.cpu().numpy())
            targets.extend(labels.cpu().numpy())
            
    accuracy = accuracy_score(targets, predictions)
    print(f'Epoch: {epoch+1}, Acc: {accuracy}')

Epoch: 1, Acc: 0.5495
